## start session

In [5]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Elasticsearch to Pyspark').getOrCreate()

25/01/29 09:24:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
from elasticsearch import Elasticsearch

es = Elasticsearch(hosts=["http://localhost:9200"])

response = es.search(index="historical", body={
    "query": {
        "match_all": {}
    },
    "size": 1000  
})

data = [hit["_source"] for hit in response["hits"]["hits"]]

In [9]:
from pyspark.sql import SparkSession

# Spark Session erstellen
spark = SparkSession.builder.appName("Elasticsearch to PySpark").getOrCreate()

training_df = spark.createDataFrame(data)


#from pyspark.ml.feature import VectorAssembler
#assembler = VectorAssembler(inputCols=["feature1", "feature2"], outputCol="features")
#dataset = assembler.transform(df).select("features", "label")

In [10]:
training_df.head()

Row(price='95146.52000000', symbol='btcusdt', time='2024-12-29T11:42:13.653000')

## read data

In [20]:
#training=spark.read.csv('/home/ubuntu/CryptoBot/regression/SalaryData.csv',  header=True, inferSchema=True, nullValue="NULL")

In [11]:
training_df.show()

+--------------+-------+--------------------+
|         price| symbol|                time|
+--------------+-------+--------------------+
|95146.52000000|btcusdt|2024-12-29T11:42:...|
|95146.52000000|btcusdt|2024-12-29T11:42:...|
|95147.64000000|btcusdt|2024-12-29T11:42:...|
|95147.64000000|btcusdt|2024-12-29T11:42:...|
|95147.84000000|btcusdt|2024-12-29T11:42:...|
|95147.86000000|btcusdt|2024-12-29T11:42:...|
|95147.86000000|btcusdt|2024-12-29T11:42:...|
|95147.84000000|btcusdt|2024-12-29T11:42:...|
|95147.83000000|btcusdt|2024-12-29T11:42:...|
|95147.63000000|btcusdt|2024-12-29T11:42:...|
|95147.87000000|btcusdt|2024-12-29T11:42:...|
|95147.89000000|btcusdt|2024-12-29T11:42:...|
|95147.86000000|btcusdt|2024-12-29T11:42:...|
|95147.82000000|btcusdt|2024-12-29T11:42:...|
|95147.87000000|btcusdt|2024-12-29T11:43:...|
|95147.89000000|btcusdt|2024-12-29T11:43:...|
|95147.92000000|btcusdt|2024-12-29T11:43:...|
|95148.75000000|btcusdt|2024-12-29T11:43:...|
|95148.75000000|btcusdt|2024-12-29

In [12]:
training_df.printSchema()

root
 |-- price: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- time: string (nullable = true)



In [13]:
training_df.columns

['price', 'symbol', 'time']

In [38]:
from pyspark.sql.functions import col, to_timestamp, unix_timestamp

training_df = training_df.withColumn("time", to_timestamp(col("time")))
training_df = training_df.withColumn("time_numeric", unix_timestamp(col("time")))

training_df = training_df.withColumn("price", col("price").cast("double"))

In [39]:
training_df.show(2)
training_df.printSchema()

+--------+-------+--------------------+------------+
|   price| symbol|                time|time_numeric|
+--------+-------+--------------------+------------+
|95146.52|btcusdt|2024-12-29 11:42:...|  1735472533|
|95146.52|btcusdt|2024-12-29 11:42:...|  1735472534|
+--------+-------+--------------------+------------+
only showing top 2 rows

root
 |-- price: double (nullable = true)
 |-- symbol: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- time_numeric: long (nullable = true)



In [40]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["time_numeric"],outputCol="Independent Features")


In [41]:
output=featureassembler.transform(training_df)

In [42]:
output.show(5)

+--------+-------+--------------------+------------+--------------------+
|   price| symbol|                time|time_numeric|Independent Features|
+--------+-------+--------------------+------------+--------------------+
|95146.52|btcusdt|2024-12-29 11:42:...|  1735472533|     [1.735472533E9]|
|95146.52|btcusdt|2024-12-29 11:42:...|  1735472534|     [1.735472534E9]|
|95147.64|btcusdt|2024-12-29 11:42:...|  1735472537|     [1.735472537E9]|
|95147.64|btcusdt|2024-12-29 11:42:...|  1735472537|     [1.735472537E9]|
|95147.84|btcusdt|2024-12-29 11:42:...|  1735472537|     [1.735472537E9]|
+--------+-------+--------------------+------------+--------------------+
only showing top 5 rows



In [43]:
output.columns

['price', 'symbol', 'time', 'time_numeric', 'Independent Features']

In [44]:
finalized_data=output.select("Independent Features","price")

In [45]:
from pyspark.ml.regression import LinearRegression
##traintest split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol="Independent Features",labelCol="price")
regressor=regressor.fit(train_data)


25/01/29 10:16:21 WARN Instrumentation: [b5239d50] regParam is zero, which might cause numerical instability and overfitting.
25/01/29 10:16:22 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/01/29 10:16:22 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [46]:
print(f"Coefficients: {regressor.coefficients}")
print(f"Intercept: {regressor.intercept}")


Coefficients: [-1.5883647098528668]
Intercept: 2756659435.7723465


In [ ]:
regressor.save("regression_model")